<a href="https://colab.research.google.com/github/hantarex/YandexML/blob/master/test_tpu10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!unzip '/gdrive/My Drive/MachineLearning/Yandex/GTSRB_Final_Training_Images.zip' -d './'

In [0]:
import skimage
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import imgaug.augmenters as iaa
import copy

In [0]:
# if tf.__version__.split('.')[1] == '13':
#     raise Exception('This notebook is not compatible with Tensorflow 1.13, please use the previous version at https://github.com/tensorflow/tpu/blob/913cf31d85bc31541fbdafa9d0b87db71dd6dcba/tools/colab/shakespeare_with_tpu_and_keras.ipynb')

In [0]:
# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [0]:
conv_kernel_sizes = [
    [3, 3],
    [3, 3],
    [3, 2, 1],
    [3, 2, 1],
]
conv_n_maps = [
    [16, 16],
    [32, 32],
    [64, 64, 64],
    [128, 128, 128],
]
conv_n_padding = [
    ["SAME", "SAME"],
    ["SAME", "SAME"],
    ["SAME", "SAME", "SAME"],
    ["SAME", "SAME", "SAME"],
]
conv_max_pool = [
    (2, 2),
    (2, 2),
    (2, 2),
    (2, 2),
]
last_layer = 256

In [0]:
batch_gen = 20
batch_size = 100

In [0]:
import shutil

In [0]:
google_path = "/gdrive/My Drive/MachineLearning/Yandex/"

In [0]:
main_path = "./"
data_directory = main_path + "GTSRB/Final_Training/Images/"
test_directory = main_path + "GTSRB/Final_Test/Images/"
save_weight_dir = google_path + "models" + "/" + "colab15"

In [0]:
def check_files():
  exist = False
  for d in os.listdir(main_path):
    if d.endswith(".tfrecord"):
      exist = True
      break
  return exist

In [0]:
directories = [d for d in os.listdir(data_directory)]

In [0]:
labels = pd.DataFrame()

In [0]:
for d in directories:
    label_directory = os.path.join(data_directory, d)
    file_names = [os.path.join(label_directory, f) for f in os.listdir(label_directory) if f.endswith(".ppm")]
    labels_names = [os.path.join(label_directory, f) for f in os.listdir(label_directory) if f.endswith(".csv")]
    for label_file in labels_names:
        lb = pd.read_csv(label_file, sep=';')
        lb['Filename'] = lb['Filename'].map(lambda x: os.path.join(label_directory, x))
        labels = labels.append(lb, ignore_index=True)

In [0]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39209 entries, 0 to 39208
Data columns (total 8 columns):
Filename    39209 non-null object
Width       39209 non-null int64
Height      39209 non-null int64
Roi.X1      39209 non-null int64
Roi.Y1      39209 non-null int64
Roi.X2      39209 non-null int64
Roi.Y2      39209 non-null int64
ClassId     39209 non-null int64
dtypes: int64(7), object(1)
memory usage: 2.4+ MB


In [0]:
width = int(round(labels['Width'].mean()))
height = int(round(labels['Height'].mean()))

In [0]:
class GenerateTFRecord:
    def __init__(self, data, batch_img=5):
        self.data = data
        self.batch_img = batch_img
        self.seq = seq = iaa.Sequential([
            iaa.GaussianBlur((0, 3.0), name="GaussianBlur"),
            iaa.Dropout(0.02, name="Dropout"),
            iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
            iaa.Affine(rotate=(-10, 10)),
            iaa.Multiply((0.1, 1.2))
        ])

    def convert_image_folder(self, tfrecord_file_name):
        # Get all file names of images present in folder
        options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
        i=1
        file_count=0
        writer = tf.python_io.TFRecordWriter(tfrecord_file_name + "_" + str(file_count) + ".tfrecord", options=options)
        for _, x in self.data.iterrows():
            examples = self._convert_image(x)
            if i%(len(self.data)//10) == 0:
                file_count+=1
                file_name = tfrecord_file_name + "_" + str(self.batch_img) + "_" + str(file_count) + ".tfrecord";
                print("Genarating: " + file_name)
                writer.close()
                writer = tf.python_io.TFRecordWriter(file_name, options=options)
            i+=1
            for example in examples:
                writer.write(example.SerializeToString())
        writer.close()

    def _convert_image(self, x):
        examples = []
        image_data = cv2.imread(x['Filename'])
        image_str = image_data.reshape(-1)
        img_shape = image_data.shape
        examples.append(tf.train.Example(features=tf.train.Features(feature={
            'filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[x['Filename'].encode('utf-8')])),
            'image': tf.train.Feature(float_list=tf.train.FloatList(value=image_str)),
            'shape': tf.train.Feature(int64_list=tf.train.Int64List(value=img_shape)),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[x['ClassId']]))
        })))
        return examples

    def _get_label_with_filename(self, filename):
        basename = os.path.basename(filename).split('.')[0]
        basename = basename.split('_')[0]
        return self.labels[basename]

In [0]:
if check_files() is False:
  X_train, X_test = train_test_split(
      labels, test_size=0.3, random_state=42, stratify=labels['ClassId']
  )
  GenerateTFRecord(X_train, batch_img=0).convert_image_folder('images_train')
  GenerateTFRecord(X_test, batch_img=0).convert_image_folder('images_test')
#   files = [google_path + d for d in os.listdir(google_path) if d.endswith(".tfrecord") and d.startswith("images_")]
#   for file in files:
#     shutil.copy(file, main_path)

Genarating: images_train_0_1.tfrecord
Genarating: images_train_0_2.tfrecord
Genarating: images_train_0_3.tfrecord
Genarating: images_train_0_4.tfrecord
Genarating: images_train_0_5.tfrecord
Genarating: images_train_0_6.tfrecord
Genarating: images_train_0_7.tfrecord
Genarating: images_train_0_8.tfrecord
Genarating: images_train_0_9.tfrecord
Genarating: images_train_0_10.tfrecord
Genarating: images_test_0_1.tfrecord
Genarating: images_test_0_2.tfrecord
Genarating: images_test_0_3.tfrecord
Genarating: images_test_0_4.tfrecord
Genarating: images_test_0_5.tfrecord
Genarating: images_test_0_6.tfrecord
Genarating: images_test_0_7.tfrecord
Genarating: images_test_0_8.tfrecord
Genarating: images_test_0_9.tfrecord
Genarating: images_test_0_10.tfrecord


In [0]:
def create_generator(name, batch=100, batch_img=5, with_gen=True, aug=None):
    def _resize_function(image_decoded, label):
        image_decoded.set_shape([None, None, None])
        image_resized = tf.image.resize_images(image_decoded, [width, height])
        image_resized /= 255.0
        return image_resized, label
    
    def extract_fn(data_record):
        features = {
            'filename': tf.FixedLenFeature([], tf.string),
            'image': tf.VarLenFeature(tf.float32),
            'shape': tf.FixedLenFeature([3], tf.int64),
            'label': tf.FixedLenFeature([], tf.int64)
        }
        # Extract the data record
        sample = tf.parse_single_example(data_record, features)
        shape = sample['shape']
        image = tf.sparse.to_dense(sample['image'])
        image = tf.reshape(image, shape)
        label = sample['label']
        return image, label

    files = [main_path + d for d in os.listdir(main_path) if d.endswith(".tfrecord") and d.startswith(name)]
    print(files)
    dataset = tf.data.TFRecordDataset(files, compression_type="GZIP")
    dataset = dataset.map(extract_fn)
    dataset = dataset.map(_resize_function)
    if aug is not None:
        dataset = dataset.map(lambda x, y: (
            tf.py_func(lambda x: aug.augment_images(x), [x], tf.float32),
            y
        ))
    shapes = (tf.TensorShape([width, height, 3]), tf.TensorShape([]))
    dataset = dataset.apply(tf.contrib.data.assert_element_shape(shapes))
    dataset = dataset.batch(batch)
    return dataset

In [0]:
seq = iaa.Sequential([
    iaa.GaussianBlur((0, 3.0), name="GaussianBlur"),
    iaa.Dropout(0.02, name="Dropout"),
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
    iaa.Affine(rotate=(-10, 10)),
    iaa.Multiply((0.05, 1.2))
])

In [0]:
dataset = create_generator(name="images_train_0", batch_img=batch_gen, batch=batch_size)
dataset_test = create_generator(name="images_test_0", batch_img=batch_gen, batch=1)

['./images_train_0_6.tfrecord', './images_train_0.tfrecord', './images_train_0_1.tfrecord', './images_train_0_2.tfrecord', './images_train_0_4.tfrecord', './images_train_0_8.tfrecord', './images_train_0_5.tfrecord', './images_train_0_10.tfrecord', './images_train_0_9.tfrecord', './images_train_0_7.tfrecord', './images_train_0_3.tfrecord']


W0617 11:04:18.507228 140324689704832 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



['./images_test_0_9.tfrecord', './images_test_0_1.tfrecord', './images_test_0_2.tfrecord', './images_test_0_7.tfrecord', './images_test_0_6.tfrecord', './images_test_0_8.tfrecord', './images_test_0_3.tfrecord', './images_test_0_4.tfrecord', './images_test_0_5.tfrecord', './images_test_0.tfrecord', './images_test_0_10.tfrecord']


In [0]:
def augment(x, y):
    return tf.py_func(lambda x: seq.augment_images(x), [x], tf.float32), y

In [0]:
dataset_aug = [dataset.map(augment) for _ in range(batch_gen)]

W0617 11:04:18.614043 140324689704832 deprecation.py:323] From <ipython-input-22-bbc1163ffa5c>:2: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [0]:
for d in dataset_aug: dataset = dataset.concatenate(d)

In [0]:
dataset

<DatasetV1Adapter shapes: (<unknown>, (?,)), types: (tf.float32, tf.int64)>

In [0]:
shapes = (tf.TensorShape([None, width, height, None]), tf.TensorShape([None]))
dataset = dataset.apply(tf.contrib.data.assert_element_shape(shapes))

In [0]:
dataset = dataset.cache(main_path + "/cache").shuffle(1000)

In [0]:
def model(name):
    inputs = tf.keras.Input(shape=(width, height, 3))
    prev_layer = inputs
    for num_conv, _ in enumerate(conv_kernel_sizes):
        concat = tf.keras.layers.Conv2D(
            filters=conv_n_maps[num_conv][-1],
            kernel_size=conv_kernel_sizes[num_conv][-1],
            padding=conv_n_padding[num_conv][-1],
            activation="relu"
        )(prev_layer)
        # concat = tf.keras.layers.MaxPool2D(conv_max_pool[num_conv])(concat)
        for num_map, _ in enumerate(conv_n_maps[num_conv]):
            prev_layer = tf.keras.layers.Conv2D(
                filters=conv_n_maps[num_conv][num_map],
                kernel_size=conv_kernel_sizes[num_conv][num_map],
                padding=conv_n_padding[num_conv][num_map],
                activation="relu"
            )(prev_layer)
        prev_layer = tf.keras.layers.Add()([prev_layer, concat])
        prev_layer = tf.keras.layers.BatchNormalization()(prev_layer)
        prev_layer = tf.keras.layers.MaxPool2D(conv_max_pool[num_conv])(prev_layer)
        prev_layer = tf.keras.layers.Dropout(0.25)(prev_layer)

    output_m = tf.keras.layers.Flatten()(prev_layer)
    pred = tf.keras.layers.Dense(last_layer, kernel_initializer="glorot_uniform", activation=tf.nn.relu)(output_m)
    pred = tf.keras.layers.BatchNormalization()(pred)
    pred = tf.keras.layers.Dropout(0.5)(pred)
    pred = tf.keras.layers.Dense(43, kernel_initializer="glorot_uniform", activation='softmax')(pred)
    model_train = tf.keras.Model(inputs=inputs, outputs=pred, name=name)
    model_train.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(1e-3),
        metrics=['sparse_categorical_accuracy']
    )
    model_train.summary()
    return model_train

In [0]:
# tf.keras.backend.clear_session()

# resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
# tf.contrib.distribute.initialize_tpu_system(resolver)
# strategy = tf.contrib.distribute.TPUStrategy(resolver)

In [0]:
# with strategy.scope():
model = model("test")

W0617 11:04:18.975279 140324689704832 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 51, 50, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 51, 50, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 51, 50, 16)   2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 51, 50, 16)   448         input_1[0][0]                    
_______________________________________________________________________________________________

In [0]:
if os.path.isfile(save_weight_dir + ".index"):
    print("Load weights!")
    model.load_weights(save_weight_dir)

In [0]:
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        self.model.save_weights(save_weight_dir + "_epoch")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('Restoring model weights from the end of the best epoch.')
                self.model.set_weights(self.best_weights)
                self.model.save_weights(save_weight_dir)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [34]:
model.fit(
    dataset,
    epochs=30,
    callbacks=[
        EarlyStoppingAtMinLoss()
    ],
)

Epoch 1/30
5775/5775 [==============================] - 200s 35ms/step - loss: 0.0115 - sparse_categorical_accuracy: 0.9966
Epoch 2/30
5775/5775 [==============================] - 191s 33ms/step - loss: 0.0108 - sparse_categorical_accuracy: 0.9968
Epoch 3/30
5775/5775 [==============================] - 192s 33ms/step - loss: 0.0108 - sparse_categorical_accuracy: 0.9967
Epoch 00003: early stopping


In [35]:
model.evaluate(dataset_test)

  11763/Unknown - 65s 5ms/step - loss: 0.0058 - sparse_categorical_accuracy: 0.9987

[0.005774125717965833, 0.9987248]